# Q1: Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

/Users/ivanivanov/miniconda3/envs/pdf_chats/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ivanivanov/miniconda3/envs/pdf_chats/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [20]:
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)
v[0]

0.07822261

### Prepare docs

In [6]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
all_documents = docs_response.json()

In [8]:
# docs where course is 'machine-learning-zoomcamp'
documents = [doc for doc in all_documents if doc['course'] == 'machine-learning-zoomcamp']
len(documents)

375

# Q2: Creating the embeddings

In [9]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [15]:
import numpy as np

embeddings = []

for doc in documents:
    qa_text = doc['question'] + ' ' + doc['text']
    enc_qa = embedding_model.encode(qa_text)
    embeddings.append(enc_qa)

X = np.array(embeddings)
X.shape

(375, 768)

# Q3: Search

In [21]:
v @ v # dot product = 1 ~ vectors already normalised

0.9999999

In [22]:
scores = X.dot(v)

In [23]:
max(scores)

0.6506574

### Vector search

In [24]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

# Q4: Hit-rate for our search engine

In [25]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [42]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [102]:
from tqdm.auto import tqdm

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        q['emb_question'] = embedding_model.encode(q['question'])
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total)
    }

In [50]:
evaluate(ground_truth, lambda q: search_engine.search(q['emb_question'], num_results=5))
# 0.93

100%|██████████| 1830/1830 [00:44<00:00, 41.05it/s]


{'hit_rate': 0.9398907103825137}

# Q5: Indexing with Elasticsearch

In [64]:
# create index
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "emb_question": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [65]:
# Index the embeddings
for doc, emb in zip(documents, X):
    doc['emb_question'] = emb.tolist()
    es_client.index(index=index_name, id=doc['id'], body=doc)

In [73]:
# let's perform the search of the same query from Q1.
# What's the ID of the document with the highest score?
query = {
    "query": {
        "script_score": {
            "query": {
                "match_all": {}
            },
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'emb_question') + 1.0",
                "params": {
                    "query_vector": v.tolist()
                }
            }
        }
    }
}

res = es_client.search(index=index_name, body=query)
res['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'ee58a693',
  '_score': 1.6506574,
  '_source': {'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
   'section': 'General course-related questions',
   'question': 'The course has already started. Can I still join it?',
   'course': 'machine-learning-zoomcamp',
   'id': 'ee58a693',
   'emb_question': [0.08062857389450073,
    -0.06663883477449417,
    0.02527310885488987,
    -0.01300442311912775,
    0.07587287575006485,
    -0.05946263670921326,
    -0.021883904933929443,
    0.0029000616632401943,
    0.0007929405546747148,
    -0.0052223606035113335,
    -0.03365178406238556,
    -0.027913514524698257,
    0.0581

In [100]:
def elastic_search(vector):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "match_all": {}
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'emb_question') + 1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ]
            }
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source']) # hit['_score']
 
    return result_docs

In [91]:
elastic_search(v.tolist())

[({'question': 'The course has already started. Can I still join it?',
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
   'id': 'ee58a693'},
  1.6506574),
 ({'question': 'I just joined. What should I do next? How can I access course materials?',
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and

# Q6: Hit-rate for Elasticsearch

In [93]:
def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [94]:
evaluate(ground_truth, lambda q: elastic_search_knn('emb_question', q['emb_question']))

100%|██████████| 1830/1830 [00:34<00:00, 52.51it/s]


{'hit_rate': 0.9398907103825137}

In [103]:
evaluate(ground_truth, lambda q: elastic_search(q['emb_question']))

100%|██████████| 1830/1830 [00:33<00:00, 54.28it/s]


{'hit_rate': 0.9398907103825137}

In [104]:
def elastic_search_knn_combined(vector):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "match_all": {}
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'emb_question') + 1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ]
            }
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [105]:
evaluate(ground_truth, lambda q: elastic_search_knn_combined(q['emb_question']))

100%|██████████| 1830/1830 [00:34<00:00, 52.96it/s]


{'hit_rate': 0.9398907103825137}